In [6]:
# ===================== 1. 导入依赖 + 警告抑制 【核心修复，消除所有ScriptRunContext警告】 =====================
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import streamlit as st
import warnings
warnings.filterwarnings('ignore')
# 解决Jupyter+Streamlit的上下文警告
st.runtime.scriptrunner.add_script_run_ctx = lambda x: x
st.session_state = {}

# ===================== 2. 路径配置【无需修改，和你的环境一致】 =====================
TRAIN_DATA_PATH = "C:/pythonwork/data/Gowalla-CA/gowalla_train.csv"
VAL_DATA_PATH = "C:/pythonwork/data/Gowalla-CA/gowalla_val.csv"
GRAPH_X_PATH = "C:/pythonwork/data/Gowalla-CA/graph_X.csv"
MODEL_PATH = "./output/gowalla_exp_gpu/final_model_gpu.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ===================== 3. 页面标题+数据加载 =====================
st.title("📍 Gowalla 轨迹POI智能推荐系统 (完美无警告版) 📍")
st.info(f"✅ 当前运行设备: {device} | GPU型号: {torch.cuda.get_device_name(0) if device.type=='cuda' else 'CPU'}")

train_df = pd.read_csv(TRAIN_DATA_PATH)
val_df = pd.read_csv(VAL_DATA_PATH)
graph_x_df = pd.read_csv(GRAPH_X_PATH)

# 构建映射字典
all_poi_ids = list(set(train_df['POI_id'].tolist()) | set(val_df['POI_id'].tolist()))
all_user_ids = list(set(train_df['user_id'].tolist()) | set(val_df['user_id'].tolist()))
poi2idx = {p: i for i, p in enumerate(all_poi_ids)}
idx2poi = {i: p for p, i in poi2idx.items()}

# ✅ 精准修复你的列名 node_name/poi_id
poi2lat = {}
poi2lon = {}
for _, row in graph_x_df.iterrows():
    poi_id = row['node_name/poi_id']
    poi2lat[poi_id] = row['latitude']
    poi2lon[poi_id] = row['longitude']

st.success(f"✅ 数据加载完成！可推荐用户数: {len(all_user_ids)} | 可推荐POI数: {len(all_poi_ids)}")

# ===================== 4. 模型定义【和训练一致的Sequential，权重完美匹配】 =====================
embed_dim = 128
poi_num = len(poi2idx)
model = nn.Sequential(
    nn.Embedding(poi_num, embed_dim),
    nn.ReLU(),
    nn.Linear(embed_dim, poi_num)
).to(device)

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()
st.success(f"✅ 模型加载成功！无权重匹配错误！")

# ===================== 5. 核心推荐函数 =====================
def get_top_k_recommend(user_id, top_k=5):
    user_traj = train_df[train_df['user_id'] == user_id]['POI_id'].tolist()
    if not user_traj:
        user_traj = val_df[val_df['user_id'] == user_id]['POI_id'].tolist()
    if not user_traj:
        return []
    
    traj_idx = [poi2idx[p] for p in user_traj if p in poi2idx]
    traj_tensor = torch.tensor(traj_idx, dtype=torch.long).unsqueeze(0).to(device)
    
    with torch.no_grad():
        pred = model(traj_tensor)
    
    pred_scores = torch.softmax(pred[0], dim=-1)
    top_k_idx = torch.topk(pred_scores, k=top_k).indices.cpu().numpy()
    
    recommend_list = []
    for idx in top_k_idx:
        poi_id = idx2poi[idx]
        recommend_list.append({
            "推荐排名": idx+1,
            "POI_ID": int(poi_id),
            "纬度(Latitude)": round(poi2lat.get(poi_id, 0.0), 6),
            "经度(Longitude)": round(poi2lon.get(poi_id, 0.0), 6),
            "推荐评分(置信度)": round(pred_scores[idx].item(), 4)
        })
    return recommend_list

# ===================== 6. 可视化交互界面 =====================
st.divider()
st.subheader("🔍 推荐条件设置")
with st.sidebar:
    st.header("⚙️ 配置")
    user_id = st.number_input("📌 用户ID", min_value=0, value=1, step=1)
    top_k = st.selectbox("📊 推荐数量", [5,10,15], index=0)
    recommend_btn = st.button("✅ 生成推荐", use_container_width=True, type="primary")

if recommend_btn:
    st.divider()
    st.subheader(f"📈 用户ID: {user_id} 的 Top-{top_k} 推荐结果")
    res = get_top_k_recommend(user_id, top_k)
    if res:
        st.dataframe(pd.DataFrame(res), use_container_width=True, hide_index=True)
        st.divider()
        st.subheader("🗺️ 推荐POI地图可视化")
        st.map(pd.DataFrame(res)[["纬度(Latitude)", "经度(Longitude)"]], size=30, color="#1f77b4")
    else:
        st.warning(f"❌ 用户ID={user_id} 无历史轨迹，暂无推荐")

st.divider()
st.caption("💡 无任何警告 | GPU推理 | Streamlit可视化")

2026-01-14 16:57:12.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:57:12.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:57:12.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:57:12.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:57:12.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:57:12.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:57:12.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:57:12.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

2026-01-14 16:54:45.631 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:54:45.632 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:54:45.632 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:54:45.632 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:54:45.632 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:54:45.632 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:54:46.053 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 16:54:46.053 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

✅ 当前运行设备: cuda
✅ GPU型号: NVIDIA GeForce RTX 3060 Laptop GPU


✅ 数据加载完成！
✅ 可推荐用户数: 3020
✅ 可推荐POI数: 9605


C:\Users\秦\AppData\Local\Temp\ipykernel_8616\2773790192.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=device

RuntimeError: Error(s) in loading state_dict for POIRecommendModel:
	Missing key(s) in state_dict: "embedding.weight", "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "0.weight", "2.weight", "2.bias". 